In [10]:
import os
import gpxpy
import pandas as pd
import geopy.distance
from dotenv import load_dotenv
from numpy_ext import rolling_apply
from sqlalchemy import create_engine, text

def rolling_metric_pace(duration, distance):
    return (duration.sum() / 60) / (distance.sum() / 1000)

notebook_path = os.path.abspath("run_uploads.ipynb")

print("Hello world")

Hello world


In [2]:
engine = create_engine(os.environ['PSQL'])

In [18]:
uploaded_files = os.listdir(".")
for file in uploaded_files:
    if "route_" in file:
        train_csv = os.path.join(os.path.dirname(notebook_path), file)
        with open(train_csv) as f:
            bytes_data = f.read()
            gpx = gpxpy.parse(bytes_data)
            points = []
            for segment in gpx.tracks[0].segments:
                for p in segment.points:
                    points.append(
                        {'timestamp': p.time, 'lat': p.latitude, 'lon': p.longitude, 'elevation': p.elevation, })
            df = pd.DataFrame.from_records(points)
            df['run_id'] = f.name.split('.')[0].split('/')[-1]
            coords = [(p.lat, p.lon) for p in df.itertuples()]
            df['distance'] = [0] + [geopy.distance.distance(from_, to).m for from_, to in zip(coords[:-1], coords[1:])]
            df['distance_cumulative'] = df.distance.cumsum()
            df['duration'] = df.timestamp.diff().dt.total_seconds().fillna(0)
            df['duration_cumulative'] = df.duration.cumsum()
            df['rolling_average_pace'] = rolling_apply(rolling_metric_pace, 100, df.duration.values, df.distance.values)
            df['pace_metric'] = pd.Series((df.duration / 60) / (df.distance / 1000)).bfill()
            #'''
            try:
                with engine.connect() as con:
                    con.execute(
                        text(f"""DELETE FROM public.public_running_coordinates WHERE run_id='{f.name.split('.')[0]}' """))
                    con.commit()
                df.to_sql(f"public_running_coordinates", index=False, if_exists='append', con=engine)
            except Exception as e:
                print(e)
            #'''
            print(df)


                     timestamp        lat        lon   elevation  \
0    2026-02-11 15:30:36+00:00  27.787082 -17.931143  904.048646   
1    2026-02-11 15:30:37+00:00  27.787071 -17.931137  903.241500   
2    2026-02-11 15:30:38+00:00  27.787060 -17.931132  902.464313   
3    2026-02-11 15:30:39+00:00  27.787045 -17.931128  901.717227   
4    2026-02-11 15:30:40+00:00  27.787029 -17.931126  900.999506   
...                        ...        ...        ...         ...   
1412 2026-02-11 15:54:09+00:00  27.786758 -17.931369  905.374859   
1413 2026-02-11 15:54:10+00:00  27.786770 -17.931370  905.039993   
1414 2026-02-11 15:54:11+00:00  27.786782 -17.931372  904.641425   
1415 2026-02-11 15:54:12+00:00  27.786797 -17.931376  904.184312   
1416 2026-02-11 15:54:13+00:00  27.786812 -17.931381  903.682874   

                  run_id  distance  distance_cumulative  duration  \
0     route_2026-02-11_3  0.000000             0.000000       0.0   
1     route_2026-02-11_3  1.354832           